In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import dask.bag as db
from dask.distributed import Client
client = Client()
# check status at: http://localhost:8787/status

In [2]:
data_dir = "data/ares"

In [3]:
vr_full_name = pd.read_csv(f"{data_dir}/ares_ciselnik_VR.csv", sep=';', header=None).set_index(0)[1]
vr_full_name

0
NR                              Nadační rejstřík
OR                             Obchodní rejstřík
ROPS     Rejstřík obecně prospěšných společností
RSVJ    Rejstřík společenství vlastníků jednotek
RU                               Rejstřík ústavů
SR                             Spolkový rejstřík
Name: 1, dtype: object